# Bertopic modeling: steps and algorithms

Ref.: https://maartengr.github.io/BERTopic/algorithm/algorithm.html

Bertopic Modeling is an integrated topic modeling approach by Maarten Grootendorst based on Google's BERT. 

It can be considered as a pipeline of 5 steps (with one additional optional step). It is very modular, which means that in each step you can replace the default by your own version. 

![](img/bertopic_steps.svg)
  
1. Embeddings
- BERTopic starts with transforming our input documents into numerical representations. Although there are many ways this can be achieved, it typically uses sentence-transformers ("all-MiniLM-L6-v2") as it is quite capable of capturing the semantic similarity between documents (see [Intro to NLP](942-nlp.ipynb)).
2. Dimensionality Reduction
- As sentence embeddings are often high in dimensionality, clustering becomes difficult due to the curse of dimensionality. A solution is to reduce the dimensionality of the embeddings to a workable dimensional space (e.g., 5) for clustering algorithms to work with.
- UMAP is used as a default in BERTopic since it can capture both the local and global high-dimensional space in lower dimensions.
- However, there are other solutions out there, such as PCA that users might be interested in trying out. Since BERTopic assumes some independency between steps, we can use any other dimensionality reduction algorithm. 
3. Clustering
- After reducing the dimensionality of our input embeddings, we need to cluster them into groups of similar embeddings to extract our topics. 
- In BERTopic, we typically use HDBSCAN as it is quite capable of capturing structures with different densities. 
- One of the main differences between DBSCAN and HDBSCAN is the way they identify clusters. 
  - DBSCAN uses a density-based approach, where a cluster is defined as a dense region of points that is separated from other dense regions by areas of lower point density. 
  - DBSCAN requires two parameters, a distance metric (ε), and a density threshold (min_samples), to identify clusters. The distance metric is used to determine the similarity between points, while the density threshold is used to define the minimum number of points required to form a cluster (see [Notebook Clustering](910-unsupervised_learning.ipynb) ).
  - HDBSCAN, on the other hand, uses a hierarchical density-based approach, where clusters are identified by recursively merging smaller clusters of points that are adjacent to each other. 
  - This approach allows HDBSCAN to identify clusters of varying densities and shapes, as well as subclusters within those clusters. 
  - HDBSCAN does not require any parameters to be set, making it even more flexible than DBSCAN.
- However, there is not one perfect clustering model and you might want to be using something entirely different for your use case.
4. Tokenizer  
- After clustering we have to find a good representation of the topics, based on the most frequent words in each cluster. Sklearn's `CountVectorizer` is used  here, as it counts the frequency of each word in each document. 
- Since it's merely representation, it can be used after clustering (or topic modeling) to update the topics by using ngrams (groups of consecutive words) and specifying the minimum document frequency:
    
  ```
  from sklearn.feature_extraction.text import CountVectorizer
  vectorizer_model = CountVectorizer(ngram_range=(1, ngram), stop_words="english", min_df=8)  
  # min_df is  an integer representing how frequent a word must be before being added to the representation.
  topic_model.update_topics(art['text'], vectorizer_model=vectorizer_model)
  ```
5. Weighting Scheme  
  
- Next, the topics that are identified in the previous step are weighted with respect to the number of clusters (or classes) in which they appear. That means, a word (or n-gram) that occurs in almost all classes, will receive a lower importance than one that appears in only one or two classes. TF-IDF is used (see [Intro to NLP](942-nlp.ipynb)) here, but iso of document frequency here class frequency is used, hence c-Tf-IDF.

6. Representation Tuning  
  
- As this step is optional, let's work here with what is proposed by the author: 
    
  ```
  from bertopic.representation import KeyBERTInspired
  representation_model = KeyBERTInspired()
  ```